In [1]:
import numpy as np
import torch
import torch.optim as optim
import os
from torch.distributions import Categorical
from pathlib import Path
import glob
import gc

from mctsnet import MCTSnet
from tangram import Tangram
from mctsnettree import MCTSnetTree
from itertools import count
%matplotlib inline

In [2]:
torch.set_default_dtype(torch.float64)
torch.autograd.set_detect_anomaly(True)

gamma=0.9
seed=543
render=False
log_interval=1
gpu=True
load_agent=True

n_grid = 10
n_blocks = 3
n_possible_blocks = 5

embedding_size = 8
hidden_size = 8
action_dims = [n_possible_blocks,n_grid,n_grid]
state_dims = [2,n_grid,n_grid]
embedding_n_residual_blocks = 3
embedding_channel_sizes = [4,4,4,2]
embedding_kernels = [3,3,3,1]
embedding_strides = [1,1,1,1]
policy_n_residual_blocks = 2
policy_channel_sizes = [2,2,2,2]
policy_kernels = [3,3,3,1]
policy_strides = [1,1,1,1]

n_simuls = 10 #25

serialization_path = './models'
print('serialization_path: ',serialization_path)
serialize_every_n_episodes = 10000
# create folder 
Path(serialization_path).mkdir(parents=True, exist_ok=True)

serialization_path:  ./models


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if gpu and torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
else:
    torch.manual_seed(seed)
    device = "cpu"
    if gpu:
        gpu = False
torch.set_num_threads(os.cpu_count() - 1)
print("Running on Device = ", device)
filesave_paths = sorted(glob.glob(f'{serialization_path}/mctsnet_e*'))
if load_agent and len(filesave_paths) > 0:
    mctsnet = torch.load(open(filesave_paths[-1],'rb'), map_location=torch.device('cpu'))
    n_episodes = int(filesave_paths[-1][18:23])
    running_reward = float(filesave_paths[-1][25:].replace('.pt',''))
    print('Loaded MCTSnet from '+ filesave_paths[-1])
else:
    mctsnet = MCTSnet(embedding_size,
                      hidden_size,
                      action_dims,
                      state_dims,
                      embedding_n_residual_blocks,
                      embedding_channel_sizes,
                      embedding_kernels,
                      embedding_strides,
                      policy_n_residual_blocks,
                      policy_channel_sizes,
                      policy_kernels,
                      policy_strides,
                      device).to(device)
    n_episodes = 0
    running_reward = -1
    print('Initialized new MCTSnet')
seed += n_episodes
env = Tangram(seed, n_grid, n_blocks, n_possible_blocks)
tree = MCTSnetTree(env, embedding_size, device)
optimizer = optim.Adam(mctsnet.parameters(), lr=1e-1)
#optimizer = torch.optim.SGD(mctsnet.parameters(), lr=5e-4)

Running on Device =  cpu
Loaded MCTSnet from ./models/mctsnet_e65883_p0.43762021095769044.pt


In [4]:
def select_action(tree, n_simuls, mctsnet):
    probs, probs_mask = mctsnet(tree, n_simuls)    
    probs = torch.squeeze(probs)
    probs_mask = torch.tensor(probs_mask).to(device)
    masked_probs = probs*probs_mask 
    if ~np.any(masked_probs.clone().cpu().detach().numpy()):
        masked_probs += probs_mask
    sample_probs = masked_probs/torch.sum(masked_probs)
    m_sample = Categorical(sample_probs)
    action = m_sample.sample()
    action_id = action.item()
    block = action_id//n_grid//n_grid
    y = (action_id - block*n_grid*n_grid)//n_grid
    x = action_id - block*n_grid*n_grid - y*n_grid
    env_action = np.array([block,y,x])
    train_probs = sample_probs + torch.finfo(torch.float32).eps*(1-probs_mask)
    train_probs /= torch.sum(train_probs)
    m_train = Categorical(train_probs)
    mctsnet.update_saved_log_probs(m_train.log_prob(action))
    return env_action, action_id, torch.unsqueeze(probs, 0)

In [5]:
def finish_episode(mctsnet, optimizer):
    R = 0
    mctsnet_loss = []
    returns = []
    for r in mctsnet.get_rewards()[::-1]:
        R = r + gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns).to(device)
    for log_prob, R in zip(mctsnet.get_saved_log_probs(), returns):
        mctsnet_loss.append(torch.unsqueeze(-log_prob*R,0))
    optimizer.zero_grad()
    mctsnet_loss = torch.cat(mctsnet_loss).sum()
    mctsnet_loss.backward()
    #torch.nn.utils.clip_grad_norm_(mctsnet.parameters(), 1.0)
    optimizer.step()
    mctsnet.update_losses(mctsnet_loss.clone().cpu().detach().numpy())
    mctsnet.delete_rewards()
    mctsnet.delete_saved_log_probs()

In [6]:
reward_threshold = 0.99
for i_episode in count(n_episodes+1):
    env.reset()
    if render:
        env.render()
    tree = MCTSnetTree(env, embedding_size, device)
    ep_reward = 0
    for t in range(1, n_blocks + 1):
        env_action, action, probs = select_action(tree, n_simuls, mctsnet)
        env_state, env_reward, done = env.step(env_action)
        state = torch.unsqueeze(torch.tensor(env_state[:2]), 0).to(device)
        reward = torch.unsqueeze(torch.tensor([env_reward]), 0).to(device)
        for (child_id, child) in tree.get_root().get_children():
            if child_id == action:
                child.set_state(state)
                child.set_reward(reward)
                child.set_done(done)
                child.set_probs(probs)
                child.set_probs_mask(tree.get_env().get_mask())
                tree.set_root(child)
                break
        if render:
            env.render()
        mctsnet.update_rewards(reward)
        ep_reward += env_reward
        if done:
            break

    running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    finish_episode(mctsnet, optimizer)

    if i_episode % log_interval == 0:
        print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}\tLast loss: {:.2f}'.format(
                i_episode, ep_reward, running_reward, mctsnet.get_last_episode_loss()))
    if serialize_every_n_episodes > 0 and i_episode % serialize_every_n_episodes == 0:
        torch.save(mctsnet, f"{serialization_path}/mctsnet_e{str(i_episode).zfill(5)}_p{running_reward}.pt")
        print("Saved the model!")
        del mctsnet, optimizer
        gc.collect()
        torch.cuda.empty_cache()
        filesave_paths = sorted(glob.glob(f'{serialization_path}/mctsnet_e*'))
        mctsnet = torch.load(open(filesave_paths[-1],'rb'))
        n_episodes = int(filesave_paths[-1][18:23])
        running_reward = float(filesave_paths[-1][25:].replace('.pt',''))
        print('Loaded MCTSnet from '+ filesave_paths[-1])
        optimizer = optim.Adam(mctsnet.parameters(), lr=1e-1)
        #optimizer = torch.optim.SGD(mctsnet.parameters(), lr=5e-4)
    if running_reward > reward_threshold:
        print("Solved! Running reward is now {} and "
                "the last episode runs to {} time steps!".format(running_reward, t))
        torch.save(mctsnet, f"{serialization_path}/mctsnet_e{str(i_episode).zfill(5)}_p{running_reward}.pt")
        print("Saved the model!")
        break
    tree.delete()
    del tree
    gc.collect()
    torch.cuda.empty_cache()

Episode 65884	Last reward: 1.00	Average reward: 0.47	Last loss: 0.00
Episode 65885	Last reward: -1.00	Average reward: 0.39	Last loss: -0.00
Episode 65886	Last reward: 1.00	Average reward: 0.42	Last loss: 0.00
Episode 65887	Last reward: 1.00	Average reward: 0.45	Last loss: 0.14
Episode 65888	Last reward: -1.00	Average reward: 0.38	Last loss: -0.00
Episode 65889	Last reward: -1.00	Average reward: 0.31	Last loss: -0.00
Episode 65890	Last reward: 1.00	Average reward: 0.34	Last loss: 0.00
Episode 65891	Last reward: 1.00	Average reward: 0.38	Last loss: 0.00
Episode 65892	Last reward: 1.00	Average reward: 0.41	Last loss: 0.01
Episode 65893	Last reward: 1.00	Average reward: 0.44	Last loss: 0.00
Episode 65894	Last reward: 1.00	Average reward: 0.47	Last loss: 0.00
Episode 65895	Last reward: 1.00	Average reward: 0.49	Last loss: 0.00
Episode 65896	Last reward: 1.00	Average reward: 0.52	Last loss: 0.00
Episode 65897	Last reward: -1.00	Average reward: 0.44	Last loss: -0.00
Episode 65898	Last reward:

In [ ]:
torch.save(mctsnet, f"{serialization_path}/mctsnet_e{str(i_episode).zfill(5)}_p{running_reward}.pt")

NameError: name 'torch' is not defined